# Preparing the target tif for the machine learning

### Check the tif files in the folder

Description of the fils in the folder
- Folder : dev_test
- Two type of tif : nicfi and sentinel
- File name format
 - nicfi :  xxxx-YYYY-MM-nicfi.tif   xxxx means the index from 1 digit to 4 digits
 - sentinel : xxxx-YYYYMMDD-sentinel.tif xxxx means the index from 1 digit to 4 digits

- index: the index means they shared same index number index 


In [5]:
### Check the tif files in the folder

# Folder : dev_test
# Two type of tif : nicfi and sentinel
# File name format 
# - nicfi :  xxxx-YYYY-MM-nicfi.tif  xxxx means the index from 1 digit to 4 digits
# - sentinel : xxxx-YYYYMMDD-sentinel.tif xxxx means the index from 1 digit to 4 digits
# index: the index means they shared same index number index

import os
import re
import pandas as pd

# set the path to the folder
folder_path = r'G:\GithubProject\GoogleEarthEngineTask\Tif_ML\dev_test'

# get the list of files in the folder
list_of_files = os.listdir(folder_path)

# create two empty lists to store the file names
list_of_nicfi = []
list_of_sentinel = []

# loop through the files and separate the two types of tif files
for file in list_of_files:
    if re.search(r'nicfi.tif$', file):
        list_of_nicfi.append(file)
    elif re.search(r'sentinel.tif$', file):
        list_of_sentinel.append(file)

# create two empty lists to store the months
list_of_nicfi_months = []
list_of_sentinel_months = []

# loop through the files and extract the months
for file in list_of_nicfi:
    match = re.search(r'-(\d{4})-(\d{2})-', file)
    if match:
        year, month = match.groups()
        list_of_nicfi_months.append(f"{year}-{int(month)}")

for file in list_of_sentinel:
    match = re.search(r'-(\d{4})(\d{2})\d{2}-', file)
    if match:
        year, month = match.groups()
        list_of_sentinel_months.append(f"{year}-{int(month)}")

# create a dataframe with the months and the count of each type of tif files
all_months = sorted(set(list_of_nicfi_months + list_of_sentinel_months))
df = pd.DataFrame({
    'Month': all_months,
    'Count of Nicfi Image': [list_of_nicfi_months.count(x) for x in all_months],
    'Count of Sentinel Image': [list_of_sentinel_months.count(x) for x in all_months]
})

# print the dataframe
print(df.to_string(index=False))


 Month  Count of Nicfi Image  Count of Sentinel Image
2024-3                  1212                     3636
2024-4                  1212                     3636
2024-5                  1212                     3636


In [4]:
# remove all the files which the date are not the 2024 03 or 2024 04 or 2024 05, which means only the the 3 month data in the folder could be kept and both the nicfi and sentinel files are kept
def filter_tif_files(folder_path):
    """
    Remove TIF files that are not from March, April, or May 2024.
    Handles both NICFI (xxxx-YYYY-MM-nicfi.tif) and Sentinel (xxxx-YYYYMMDD-sentinel.tif) formats.
    
    Args:
        folder_path (str): Path to the folder containing TIF files
        
    Returns:
        tuple: (num_kept, num_removed) - Count of files kept and removed
    """
    kept = 0
    removed = 0
    
    for file in os.listdir(folder_path):
        should_keep = False
        
        if 'nicfi.tif' in file:
            # Handle NICFI format: xxxx-YYYY-MM-nicfi.tif
            match = re.search(r'-(\d{4})-(\d{2})-', file)
            if match:
                year, month = match.groups()
                should_keep = (year == '2024' and month in ['03', '04', '05'])
                
        elif 'sentinel.tif' in file:
            # Handle Sentinel format: xxxx-YYYYMMDD-sentinel.tif
            match = re.search(r'-(\d{4})(\d{2})\d{2}-', file)
            if match:
                year, month = match.groups()
                should_keep = (year == '2024' and month in ['03', '04', '05'])
        
        if not should_keep:
            os.remove(os.path.join(folder_path, file))
            removed += 1
        else:
            kept += 1
                    
    return kept, removed

# Example usage:
folder_path = r'G:\GithubProject\GoogleEarthEngineTask\Tif_ML\dev_test'
kept, removed = filter_tif_files(folder_path)
print(f"Kept {kept} files")
print(f"Removed {removed} files")





Kept 14544 files
Removed 7152 files


## Show the shape file 's attribute in table 

In [10]:
# show the attribute of the shape file print the name and of the table 

import geopandas as gpd
J1V_Clip_Shapefile = 'J1V_Clip/fire_nrt_J1VC2_536460_clip.shp'

# print the table's name 


gdf = gpd.read_file(J1V_Clip_Shapefile)

# Display the column names (attributes)
print("Shapefile attributes:")
for col in gdf.columns:
    print(f"- {col}")

# Optional: Display the first few rows to see the data
print("\nFirst few rows of the data:")
print(gdf.head())



Shapefile attributes:
- LATITUDE
- LONGITUDE
- BRIGHTNESS
- SCAN
- TRACK
- ACQ_DATE
- ACQ_TIME
- SATELLITE
- INSTRUMENT
- CONFIDENCE
- VERSION
- BRIGHT_T31
- FRP
- DAYNIGHT
- geometry

First few rows of the data:
   LATITUDE  LONGITUDE  BRIGHTNESS  SCAN  TRACK   ACQ_DATE ACQ_TIME SATELLITE  \
0  -8.02880  -79.04613       338.5  0.47   0.64 2020-01-01     1924       N20   
1  -8.02713  -79.04623       338.0  0.47   0.64 2020-01-01     1924       N20   
2  -8.02687  -79.04878       335.3  0.40   0.37 2020-01-03     1848       N20   
3 -17.95334  -70.24676       296.3  0.39   0.36 2020-01-04     0554       N20   
4  -8.02684  -79.04642       336.8  0.45   0.39 2020-01-08     1854       N20   

  INSTRUMENT CONFIDENCE VERSION  BRIGHT_T31  FRP DAYNIGHT  \
0      VIIRS          n  2.0NRT       294.1  2.1        D   
1      VIIRS          n  2.0NRT       294.4  2.4        D   
2      VIIRS          n  2.0NRT       296.1  2.3        D   
3      VIIRS          n  2.0NRT       280.4  0.5        

## Generate the target tif from the cliped shape and cross with the nicfi tif range


### Generate the target tif 

Requriments:
- Original tif image folder: G:\GithubProject\GoogleEarthEngineTask\Tif_ML\dev_test
- Two parameters input: tif file and the shapefile 
- One output : the target tif in the target folder: G:\GithubProject\GoogleEarthEngineTask\Tif_ML\target

- Rule : 
  - the tif files name's format xxxx-YYYY-MM-nicfi.tif  xxxx means the index from 1 digit to 4 digits contains the time information
  - the shapefile's attribute have ACQ_DATE is the date of the point data in the shapefile 
  - overlap the shapefile and the tif image and get the points 
  - check the ACQ_DATE of the point data if it match the tif file name then keep , collect all the points overlap the tif range and have same date YYYY-MM is same then do next
  - all the points bu buffer by 100 meter radius to generate the new tif file , the pixels value is the BRIGHTNESS  of acordding to  point 


